# Overview

## Dataset

A public dataset from kaggle was used. The original can be found [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

# Setup

Import all required packages and set the base path for the datasets

In [23]:
import pandas as pd
import numpy as np
import os

In [24]:
datasetsPath = "./sentimentalAnalysisDatasets"
datasetFile = "IMDB_dataset.csv"

print("Path to dataset files:", datasetsPath)

Path to dataset files: ./sentimentalAnalysisDatasets


In [25]:
data = pd.read_csv(os.path.join(datasetsPath, datasetFile))

In [26]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [28]:
# Silence downcasting deprication warning
pd.set_option('future.no_silent_downcasting', True)

# Replace "positive" -> 1, "negative" -> 0

data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [29]:
from bs4 import BeautifulSoup
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes
                           u"\U0001F800-\U0001F8FF"  # Supplemental arrows
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Remove any and all HTML tags from the review.
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data["review"] = data["review"].apply(remove_html_tags)
data["review"] = data["review"].apply(remove_emojis)
data["review"] = data["review"].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)


C:\Users\kille\AppData\Local\Temp\ipykernel_11928\949526174.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [30]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically theres a family where a little boy J...,0
4,Petter Matteis Love in the Time of Money is a ...,1


In [31]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
(train_data, test_data) = train_test_split(data, test_size = 0.2, random_state=42)

In [34]:
train_data.shape


(40000, 2)

In [35]:
test_data.shape

(10000, 2)

In [36]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(train_data["review"])

In [37]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=400)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=400)

In [38]:
Y_train = train_data["sentiment"].astype('int64')
Y_test = test_data["sentiment"].astype('int64')

In [39]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [40]:
model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 128, input_length = 400))
model.add(LSTM(128, dropout=0.4, recurrent_dropout = 0.4))
model.add(Dense(1, activation = "sigmoid"))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 128)          1280000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [43]:
history = model.fit(X_train,
                    Y_train,
                    epochs = 10,
                    batch_size = 64,
                    steps_per_epoch = 150,
                    validation_split = 0.3)


Epoch 1/10
  2/150 [..............................] - ETA: 5:25 - loss: 0.6936 - accuracy: 0.4766

KeyboardInterrupt: 

In [ ]:
model_eval = model.evaluate(X_test, Y_test)

print(f"Test Loss: {model_eval[0]}")
print(f"Test Accuarcy: {model_eval[1]}")